<h1><center>Análisis de Sistemas de Infraestructura <br>
    Taller 2: Machine Learning </center></h1>

 Importar las librerías que usaremos en el taller

In [2]:
#Importar las librerías necesarias

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import time
from sklearn import preprocessing
import tensorflow as tf #Importacion de librerias de trabajo
from tensorflow import keras
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

<h2> 1. K-Means y regresiones lineales.</h2>

<h3> Importar los datos </h3>

In [3]:
#Cargar los datos 

GCI= pd.read_excel(r'WEF_GCI_4.0_2019_Dataset.xlsx') 

#Eliminación de columnas con información sobrante

GCI.drop(['Index','Series Global ID','Freeze date','Series units','Series order','Series code (if applicable)'], axis=1, inplace=True)

GCI #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,Edition,Series name,Series type,Attribute,Angola,Albania,United Arab Emirates,Argentina,Armenia,Australia,...,Sub-Saharan Africa,Latin America and the Caribbean,Eurasia,East Asia and Pacific,South Asia,Low-income,Lower-middle-income,Upper-middle-income,High-income,Sample average
0,2019,Global Competitiveness Index 4.0,Index,VALUE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Global Competitiveness Index 4.0,Index,RANK,136,81,25,83,69,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2019,Global Competitiveness Index 4.0,Index,SCORE,38.112486,57.614169,75.007388,57.201329,61.276876,78.746626,...,47.250966,56.748896,59.379695,69.868627,54.704605,43.74455,51.949423,59.79123,72.78081,60.639694
3,2019,Global Competitiveness Index 4.0,Index,DATE DESCRIPTION,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,...,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition,2019 edition
4,2019,Global Competitiveness Index 4.0,Index,SOURCE,"World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...",...,"World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes...","World Economic Forum, [i]Global Competitivenes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,2017 backcast,Trademark applications (per million pop.),Indicator,DATE DESCRIPTION,2014-2016 moving average,2014-2016 moving average,2014-2016 moving average,2014-2016 moving average,2014-2016 moving average,2014-2016 moving average,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3384,2017 backcast,Trademark applications (per million pop.),Indicator,SOURCE,WIPO statistics database https://www3.wipo.int...,WIPO statistics database https://www3.wipo.int...,WIPO statistics database https://www3.wipo.int...,WIPO statistics database https://www3.wipo.int...,WIPO statistics database https://www3.wipo.int...,WIPO statistics database https://www3.wipo.int...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3385,2017 backcast,Trademark applications (per million pop.),Indicator,SCORE,0,59.46941022499467,82.20665106874179,76.65792838143709,68.74667858910165,88.99118432145029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3386,2017 backcast,Trademark applications (per million pop.),Indicator,VALUE,0,250.66358915176,2082.23472092856,1242.74833014335,595.142378559464,3913.12246507435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Filtrar Dataframe para el año 2019

GCI_2019=GCI[GCI['Edition']==2019].copy()
GCI_2019.drop(['Sample average','High-income','Upper-middle-income','Lower-middle-income','Low-income','East Asia and Pacific','Latin America and the Caribbean','Middle East and North Africa','Europe and North America','Sub-Saharan Africa','Eurasia','South Asia'], axis=1, inplace=True)

# Filtrar Dataframe para el pilar de Infraestructura

GCI_2019_Infrastructurei=GCI_2019[GCI_2019['Series name']=='2nd pillar: Infrastructure'].copy() #Selecciono la serie adecuada
GCI_2019_Infrastructurei=GCI_2019_Infrastructurei[GCI_2019_Infrastructurei['Attribute']=='SCORE'].copy() #Selecciono el atributo adecuado
GCI_2019_Infrastructurei.drop(['Edition','Series type','Attribute'], axis=1, inplace=True)
GCI_2019_Infrastructurei.set_index('Series name',inplace=True)

GCI_2019_Infrastructure=GCI_2019_Infrastructurei.transpose()
GCI_2019_Infrastructure.index.names = ['Country Name']
GCI_2019_Infrastructure.columns=['2nd pillar: Infrastructure']
GCI_2019_Infrastructure #Para revisar visualmente que esta pasando con nuestro arreglo de datos

GCI_2019_Infrastructure

,2nd pillar: Infrastructure
Country Name,
Angola,40.197009
Albania,57.693245
United Arab Emirates,88.485542
Argentina,68.292664
Armenia,69.408459
...,...
Viet Nam,65.92467
Yemen,33.854147
South Africa,68.134202


In [4]:
# Dataframe con los indicadores para el pilar de infraestructura

road_c=GCI_2019[GCI_2019['Series name']=='Road connectivity'].copy() #Selecciono indicador 
road_c=road_c[road_c['Attribute']=='SCORE'].copy() #Selecciono atributo

quality_r= GCI_2019[GCI_2019['Series name']=='Quality of road infrastructure'].copy()
quality_r=quality_r[quality_r['Attribute']=='SCORE'].copy()

railroad_d=GCI_2019[GCI_2019['Series name']=='Railroad density'].copy()
railroad_d=railroad_d[railroad_d['Attribute']=='SCORE'].copy()

train_s= GCI_2019[GCI_2019['Series name']=='Efficiency of train services'].copy()
train_s=train_s[train_s['Attribute']=='SCORE'].copy()

airport_c= GCI_2019[GCI_2019['Series name']=='Airport connectivity'].copy()
airport_c=airport_c[airport_c['Attribute']=='SCORE'].copy()

air_transport_s= GCI_2019[GCI_2019['Series name']=='Efficiency of air transport services'].copy()
air_transport_s=air_transport_s[air_transport_s['Attribute']=='SCORE'].copy()

liner_s= GCI_2019[GCI_2019['Series name']=='Liner shipping connectivity'].copy()
liner_s=liner_s[liner_s['Attribute']=='SCORE'].copy()

seaport_s= GCI_2019[GCI_2019['Series name']=='Efficiency of seaport services'].copy()
seaport_s=seaport_s[seaport_s['Attribute']=='SCORE'].copy()

electricity_s= GCI_2019[GCI_2019['Series name']=='Electricity supply quality'].copy()
electricity_s=electricity_s[electricity_s['Attribute']=='SCORE'].copy()

drinking_water= GCI_2019[GCI_2019['Series name']=='Exposure to unsafe drinking water'].copy()
drinking_water=drinking_water[drinking_water['Attribute']=='SCORE'].copy()

water_supply= GCI_2019[GCI_2019['Series name']=='Reliability of water supply'].copy()
water_supply=water_supply[water_supply['Attribute']=='SCORE'].copy()

lista_indicadores=[road_c, quality_r, railroad_d, train_s, airport_c, air_transport_s, liner_s, seaport_s, electricity_s, drinking_water, water_supply]
lista_nombres_ind=['Road connectivity','Quality of road infrastructure', 'Railroad density', 'Efficiency of train services', 'Airport connectivity','Efficiency of air transport services',  
                  'Liner shipping connectivity', 'Efficiency of seaport services', 'Electricity supply quality','Exposure to unsafe drinking water','Reliability of water supply']

for i in range(0,len(lista_indicadores)):
    lista_indicadores[i].drop(['Edition','Series type','Attribute'], axis=1, inplace=True)
    lista_indicadores[i].set_index('Series name',inplace=True)
    lista_indicadores[i]=lista_indicadores[i].transpose()
    lista_indicadores[i].index.names = ['Country Name']
    lista_indicadores[i].columns=[lista_nombres_ind[i]]
    
lista_indicadores[0] #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,Road connectivity
Country Name,
Angola,77.735153
Albania,55.523975
United Arab Emirates,90.086082
Argentina,94.531212
Armenia,58.623966
...,...
Viet Nam,63.317013
Yemen,56.669182
South Africa,96.206352


In [5]:
df_indicadores=pd.merge(lista_indicadores[0], lista_indicadores[1], on='Country Name')

for i in range(2,len(lista_indicadores)):
    df_indicadores=pd.merge(df_indicadores, lista_indicadores[i], on='Country Name')

df_indicadores #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,Road connectivity,Quality of road infrastructure,Railroad density,Efficiency of train services,Airport connectivity,Efficiency of air transport services,Liner shipping connectivity,Efficiency of seaport services,Electricity supply quality,Exposure to unsafe drinking water,Reliability of water supply
Country Name,,,,,,,,,,,
Angola,77.735153,19.168778,NaN,NaN,26.924789,38.136673,25.129999,30.359201,92.433967,23.320942,17.851432
Albania,55.523975,47.66529,38.59489,3.261648,30.083699,50.890573,3.04,54.689964,85.055987,83.396314,51.218637
United Arab Emirates,90.086082,83.428399,NaN,NaN,89.173814,83.572539,83.910004,74.272195,96.727162,90.123316,84.738501
Argentina,94.531212,43.437278,16.086038,28.127229,59.052283,57.157127,35.189999,48.203433,90.58809,93.640958,72.419961
Armenia,58.623966,44.075374,60.208993,35.065476,33.232687,60.248399,NaN,23.666414,92.60533,98.857713,69.501368
...,...,...,...,...,...,...,...,...,...,...,...
Viet Nam,63.317013,40.109336,19.0844,43.322651,86.001082,49.700852,68.82,47.311111,93.54775,66.99935,59.217032
Yemen,56.669182,18.698657,NaN,NaN,0.118317,20.587587,7.92,19.297075,81.392467,46.992395,13.7327
South Africa,96.206352,59.074656,43.181052,34.067464,63.465757,74.497286,40.110001,59.083772,95.133089,73.64035,57.257009


<h3> Procesamiento básico de los datos.</h2>

In [6]:
# A juicio del analista de de los datos: se considera que si no hay registro de indicadores relacionados a trafico ferreo, el pais no cuenta con este sistema

df_indicadores['Railroad density'] = df_indicadores['Railroad density'].fillna(0)
df_indicadores['Efficiency of train services'] = df_indicadores['Efficiency of train services'].fillna(0)
df_indicadores['Liner shipping connectivity'] = df_indicadores['Liner shipping connectivity'].fillna(0)
df_indicadores['Efficiency of seaport services'] = df_indicadores['Efficiency of seaport services'].fillna(0)
df_indicadores #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,Road connectivity,Quality of road infrastructure,Railroad density,Efficiency of train services,Airport connectivity,Efficiency of air transport services,Liner shipping connectivity,Efficiency of seaport services,Electricity supply quality,Exposure to unsafe drinking water,Reliability of water supply
Country Name,,,,,,,,,,,
Angola,77.735153,19.168778,0.000000,0.000000,26.924789,38.136673,25.129999,30.359201,92.433967,23.320942,17.851432
Albania,55.523975,47.66529,38.594890,3.261648,30.083699,50.890573,3.040000,54.689964,85.055987,83.396314,51.218637
United Arab Emirates,90.086082,83.428399,0.000000,0.000000,89.173814,83.572539,83.910004,74.272195,96.727162,90.123316,84.738501
Argentina,94.531212,43.437278,16.086038,28.127229,59.052283,57.157127,35.189999,48.203433,90.58809,93.640958,72.419961
Armenia,58.623966,44.075374,60.208993,35.065476,33.232687,60.248399,0.000000,23.666414,92.60533,98.857713,69.501368
...,...,...,...,...,...,...,...,...,...,...,...
Viet Nam,63.317013,40.109336,19.084400,43.322651,86.001082,49.700852,68.820000,47.311111,93.54775,66.99935,59.217032
Yemen,56.669182,18.698657,0.000000,0.000000,0.118317,20.587587,7.920000,19.297075,81.392467,46.992395,13.7327
South Africa,96.206352,59.074656,43.181052,34.067464,63.465757,74.497286,40.110001,59.083772,95.133089,73.64035,57.257009


In [7]:
#Eliminar los países que no contienen información de algun indicador

df_indicadores.dropna(how='any', inplace=True)

df_indicadores #Para revisar visualmente que esta pasando con nuestros arreglos de datos

,Road connectivity,Quality of road infrastructure,Railroad density,Efficiency of train services,Airport connectivity,Efficiency of air transport services,Liner shipping connectivity,Efficiency of seaport services,Electricity supply quality,Exposure to unsafe drinking water,Reliability of water supply
Country Name,,,,,,,,,,,
Angola,77.735153,19.168778,0.000000,0.000000,26.924789,38.136673,25.129999,30.359201,92.433967,23.320942,17.851432
Albania,55.523975,47.66529,38.594890,3.261648,30.083699,50.890573,3.040000,54.689964,85.055987,83.396314,51.218637
United Arab Emirates,90.086082,83.428399,0.000000,0.000000,89.173814,83.572539,83.910004,74.272195,96.727162,90.123316,84.738501
Argentina,94.531212,43.437278,16.086038,28.127229,59.052283,57.157127,35.189999,48.203433,90.58809,93.640958,72.419961
Armenia,58.623966,44.075374,60.208993,35.065476,33.232687,60.248399,0.000000,23.666414,92.60533,98.857713,69.501368
...,...,...,...,...,...,...,...,...,...,...,...
Viet Nam,63.317013,40.109336,19.084400,43.322651,86.001082,49.700852,68.820000,47.311111,93.54775,66.99935,59.217032
Yemen,56.669182,18.698657,0.000000,0.000000,0.118317,20.587587,7.920000,19.297075,81.392467,46.992395,13.7327
South Africa,96.206352,59.074656,43.181052,34.067464,63.465757,74.497286,40.110001,59.083772,95.133089,73.64035,57.257009


<h3> A partir de la base de datos de indicadores, utilice un algoritmo de clusterización (e.g., k-means)
que permita diferenciar tres niveles de desarrollo de la infraestructura. Compare el resultado con
el índice global de competitividad en infraestructura y concluya.</h3>

In [8]:
#Normalización 

Datos_pilar_infr_Array=df_indicadores.to_numpy() # Array de los indicadores
Datos_pilar_infr_Array=(Datos_pilar_infr_Array-np.min(Datos_pilar_infr_Array))/(np.max(Datos_pilar_infr_Array,axis=0)-np.min(Datos_pilar_infr_Array))       #Normalizacion
Datos_pilar_infr=pd.DataFrame(Datos_pilar_infr_Array,index=df_indicadores.index)
Datos_pilar_infr.columns = lista_nombres_ind
Datos_pilar_infr #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,Road connectivity,Quality of road infrastructure,Railroad density,Efficiency of train services,Airport connectivity,Efficiency of air transport services,Liner shipping connectivity,Efficiency of seaport services,Electricity supply quality,Exposure to unsafe drinking water,Reliability of water supply
Country Name,,,,,,,,,,,
Angola,0.777352,0.211817,0.0,0.0,0.269248,0.408937,0.2513,0.334253,0.92434,0.233209,0.180303
Albania,0.55524,0.526706,0.385949,0.03399,0.300837,0.545697,0.0304,0.602133,0.85056,0.833963,0.517318
United Arab Emirates,0.900861,0.921892,0.0,0.0,0.891738,0.896143,0.8391,0.817733,0.967272,0.901233,0.855875
Argentina,0.945312,0.479986,0.16086,0.293116,0.590523,0.612892,0.3519,0.530717,0.905881,0.93641,0.731455
Armenia,0.58624,0.487037,0.60209,0.365419,0.332327,0.64604,0.0,0.260566,0.926053,0.988577,0.701977
...,...,...,...,...,...,...,...,...,...,...,...
Viet Nam,0.63317,0.443212,0.190844,0.451468,0.860011,0.532939,0.6882,0.520893,0.935477,0.669993,0.598103
Yemen,0.566692,0.206622,0.0,0.0,0.001183,0.220759,0.0792,0.21246,0.813925,0.469924,0.138703
South Africa,0.962064,0.65278,0.431811,0.355019,0.634658,0.79883,0.4011,0.650509,0.951331,0.736404,0.578307


Realizamos K-Means...

In [12]:
kmeans = KMeans(n_clusters=3).fit(Datos_pilar_infr_Array)

centroids = kmeans.cluster_centers_

df_cluster=pd.DataFrame(kmeans.predict(Datos_pilar_infr_Array),index=Datos_pilar_infr.index, columns=['cluster'])

df_cluster_cero = df_cluster.drop(df_cluster[df_cluster['cluster']>0].index)
df_cluster_uno= df_cluster.drop(df_cluster[df_cluster['cluster']==0].index)
df_cluster_uno=df_cluster_uno.drop(df_cluster_uno[df_cluster_uno['cluster']==2].index)
df_cluster_dos= df_cluster.drop(df_cluster[df_cluster['cluster']<2].index)

df_cluster_cero #Para revisar visualmente que esta pasando con nuestro arreglo de datos

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,cluster
Country Name,
Angola,0
Albania,0
Burundi,0
Benin,0
Burkina Faso,0
Bangladesh,0
Bosnia and Herzegovina,0
Bolivia,0
Barbados,0


Para facilitarle la comparación requerida en le mostramos el siguiente algoritmo

In [16]:
Comparacion_cero=pd.merge(df_cluster_cero, GCI_2019_Infrastructure, on='Country Name', suffixes=('_left', '_right'))
Comparacion_uno=pd.merge(df_cluster_uno, GCI_2019_Infrastructure, on='Country Name', suffixes=('_left', '_right'))
Comparacion_dos=pd.merge(df_cluster_dos, GCI_2019_Infrastructure, on='Country Name', suffixes=('_left', '_right'))

In [17]:
# Organiza el DataFrame indicado de acuerdo a los datos del pilar de Infraestructura

Comparacion_cero.sort_values('2nd pillar: Infrastructure')
Comparacion_uno.sort_values('2nd pillar: Infrastructure')
Comparacion_dos.sort_values('2nd pillar: Infrastructure')

,cluster,2nd pillar: Infrastructure
Country Name,,
Ukraine,2,70.336134
Bulgaria,2,71.328401
Romania,2,71.676209
Serbia,2,73.836335
Estonia,2,75.773862
Latvia,2,76.008169
Ireland,2,76.978331
Lithuania,2,77.021884
Azerbaijan,2,77.362188


In [21]:
#Valores mínimos y máximos

max_cero=max(Comparacion_cero['2nd pillar: Infrastructure'])
min_cero=min(Comparacion_cero['2nd pillar: Infrastructure'])

max_uno=max(Comparacion_uno['2nd pillar: Infrastructure'])
min_uno=min(Comparacion_uno['2nd pillar: Infrastructure'])

max_dos=max(Comparacion_dos['2nd pillar: Infrastructure'])
min_dos=min(Comparacion_dos['2nd pillar: Infrastructure'])

In [23]:
print('El primer cluster está conformado por países con GCI en el rango', min_cero,max_cero, '\n','El segundo cluster está conformado por países con GCI en el rango', min_uno,max_uno, '\n','El tercer cluster está conformado por países con GCI en el rango', min_dos,max_dos,)

El primer cluster está conformado por países con GCI en el rango 26.878243786231074 66.85168818474732 
 El segundo cluster está conformado por países con GCI en el rango 55.554451438923365 88.48554173222215 
 El tercer cluster está conformado por países con GCI en el rango 70.33613446834686 94.33942042551192


<h3> Importar los datos de GDP del Banco Mundial </h3>

In [24]:
#Dataframe de GDP
df_GDP= pd.read_excel(r'API_NY.GDP.MKTP.CD_DS2_en_excel_v2_5871579.xls')
df_GDP.drop(['Country Code','Indicator Name','Indicator Code'], axis=1, inplace=True) #eliminar las columnas que no aportan ninguna información relevante
df_GDP.set_index('Country Name',inplace=True) #Indexar al nombre del país

df_GDP

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,NaN
Africa Eastern and Southern,2.112502e+10,2.161623e+10,2.350628e+10,2.804836e+10,2.592067e+10,2.947210e+10,3.201437e+10,3.326951e+10,3.632779e+10,4.163897e+10,...,9.859871e+11,1.006526e+12,9.273485e+11,8.851764e+11,1.021043e+12,1.007196e+12,1.000834e+12,9.275933e+11,1.081998e+12,1.169484e+12
Afghanistan,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,1.400000e+09,1.673333e+09,1.373333e+09,1.408889e+09,...,2.056449e+10,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,NaN
Africa Western and Central,1.044764e+10,1.117321e+10,1.199053e+10,1.272769e+10,1.389811e+10,1.492979e+10,1.591084e+10,1.451058e+10,1.496824e+10,1.697932e+10,...,8.339481e+11,8.943225e+11,7.686447e+11,6.913634e+11,6.848988e+11,7.670257e+11,8.225384e+11,7.864600e+11,8.444597e+11,8.778633e+11
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.334016e+11,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.067136e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.735328e+09,7.074393e+09,6.295845e+09,6.682674e+09,7.180769e+09,7.878763e+09,7.899741e+09,7.717143e+09,9.412034e+09,9.429156e+09
"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.041523e+10,4.322859e+10,4.244449e+10,3.131782e+10,2.684223e+10,2.160616e+10,NaN,NaN,NaN,NaN
South Africa,8.748597e+09,9.225996e+09,9.813996e+09,1.085420e+10,1.195600e+10,1.306899e+10,1.421139e+10,1.582139e+10,1.712479e+10,1.925699e+10,...,4.008860e+11,3.811989e+11,3.467098e+11,3.235855e+11,3.814488e+11,4.041589e+11,3.885312e+11,3.376196e+11,4.190156e+11,4.058697e+11


<h3> Realice el procesamiento de los datos de GDP. Luego, cree un DataFrame que contenga estos datos, pero solo en el año 2019 para todos los países.</h3>

In [25]:
df_GDP.dropna(how='all', inplace=True) #eliminar los paises para los cuales no existe ningún dato en la serie de tiempo
df_GDP.interpolate(method ='linear', limit_direction ='both', axis=1, inplace=True) #Interpolar
df_GDP #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,4.055866e+08,...,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,3.126019e+09
Africa Eastern and Southern,2.112502e+10,2.161623e+10,2.350628e+10,2.804836e+10,2.592067e+10,2.947210e+10,3.201437e+10,3.326951e+10,3.632779e+10,4.163897e+10,...,9.859871e+11,1.006526e+12,9.273485e+11,8.851764e+11,1.021043e+12,1.007196e+12,1.000834e+12,9.275933e+11,1.081998e+12,1.169484e+12
Afghanistan,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,1.400000e+09,1.673333e+09,1.373333e+09,1.408889e+09,...,2.056449e+10,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,1.458314e+10
Africa Western and Central,1.044764e+10,1.117321e+10,1.199053e+10,1.272769e+10,1.389811e+10,1.492979e+10,1.591084e+10,1.451058e+10,1.496824e+10,1.697932e+10,...,8.339481e+11,8.943225e+11,7.686447e+11,6.913634e+11,6.848988e+11,7.670257e+11,8.225384e+11,7.864600e+11,8.444597e+11,8.778633e+11
Angola,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,5.930503e+09,...,1.334016e+11,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.067136e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kosovo,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,5.181777e+09,...,6.735328e+09,7.074393e+09,6.295845e+09,6.682674e+09,7.180769e+09,7.878763e+09,7.899741e+09,7.717143e+09,9.412034e+09,9.429156e+09
"Yemen, Rep.",5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,5.647119e+09,...,4.041523e+10,4.322859e+10,4.244449e+10,3.131782e+10,2.684223e+10,2.160616e+10,2.160616e+10,2.160616e+10,2.160616e+10,2.160616e+10
South Africa,8.748597e+09,9.225996e+09,9.813996e+09,1.085420e+10,1.195600e+10,1.306899e+10,1.421139e+10,1.582139e+10,1.712479e+10,1.925699e+10,...,4.008860e+11,3.811989e+11,3.467098e+11,3.235855e+11,3.814488e+11,4.041589e+11,3.885312e+11,3.376196e+11,4.190156e+11,4.058697e+11


In [26]:
df_GDP=df_GDP.loc[:,'2019'] #Toma solamente la columna del año 2019

# Elimamos los datos que no son de países 
df_GDP.drop(['Arab World','Central Europe and the Baltics','Caribbean small states','East Asia & Pacific (excluding high income)',
'Early-demographic dividend','East Asia & Pacific','Europe & Central Asia (excluding high income)',
'Europe & Central Asia','Euro area','Fragile and conflict affected situations','European Union',
'High income','Heavily indebted poor countries (HIPC)','IBRD only','IDA & IBRD total','IDA total','IDA blend',
'IDA only','Latin America & Caribbean (excluding high income)','Latin America & Caribbean',
'Least developed countries: UN classification','Low income','Lower middle income','Low & middle income',
'Late-demographic dividend','Middle East & North Africa','Middle income','Middle East & North Africa (excluding high income)',
'North America','OECD members','Other small states','Pre-demographic dividend','Pacific island small states',
'Post-demographic dividend','Sub-Saharan Africa (excluding high income)','Small states',
'East Asia & Pacific (IDA & IBRD countries)','Europe & Central Asia (IDA & IBRD countries)',
'Latin America & the Caribbean (IDA & IBRD countries)','Middle East & North Africa (IDA & IBRD countries)',
'South Asia (IDA & IBRD)','Sub-Saharan Africa (IDA & IBRD countries)','Upper middle income','World'], axis=0, inplace=True)

GDP=df_GDP.to_numpy() #Crear un array a partir del DataFrame
GDP=(GDP-np.min(GDP))/(np.max(GDP,axis=0)-np.min(GDP))       #Normalizacion
df_GDP=pd.DataFrame(GDP,index=df_GDP.index) #Volvemos a contruir el DataFrame pero ahora sus datos estan normalizados
df_GDP #Para revisar visualmente que esta pasando con nuestro arreglo de datos

,0
Country Name,
Aruba,0.000156
Africa Eastern and Southern,0.046807
Afghanistan,0.000882
Africa Western and Central,0.038468
Angola,0.003239
...,...
Kosovo,0.000367
"Yemen, Rep.",0.001008
South Africa,0.018169


<h3> Para cada una de las tres categorías identificadas previamente con K-Means, construya una regresión lineal que utilice como entrada los indicadores de infraestructura y que, como resultado, encuentre el GDP </h3>

In [32]:
def caso_cero(tipo): 
        
        r_s=4
        #Caso cero
        Caso_cero=pd.merge(df_cluster_cero, Datos_pilar_infr, on='Country Name', suffixes=('_left', '_right')) #Une por el nombre del país los resultados del cluster 0 y los datos del pilar de infraestructura
        Caso_cero=pd.merge(Caso_cero, df_GDP, on='Country Name', suffixes=('_left', '_right')) #Une por el nombre del país resultados del cluster, los datos del indicador y los datos del banco mundial. 
        Caso_cero=Caso_cero.drop(['cluster'], axis=1)
        
        #Normalización
        Caso_cero_array=Caso_cero.to_numpy()
        Caso_cero_array=(Caso_cero_array-np.min(Caso_cero_array))/(np.max(Caso_cero_array,axis=0)-np.min(Caso_cero_array))       #Normalizacion
        Caso_cero=pd.DataFrame(Caso_cero_array,index=Caso_cero.index)
        
        
        list_columns=lista_nombres_ind.copy()
        list_columns.append('GDP')
        Caso_cero.columns = list_columns
        X = Caso_cero.iloc[:,0:11]
        Y = Caso_cero['GDP']
        
        #Regresión lineal
        
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state=r_s)
        lin_model = LinearRegression()
        lin_model.fit(X_train, Y_train)
        
        if tipo=='graf':
            plt.scatter(X_train['Quality of road infrastructure'],Y_train)
            plt.xlabel('Quality of road infrastructure') # Establecer el título del eje x
            plt.ylabel("GDP") # Establecer el título del eje y
            
        if tipo=='stad':
            
            # Evaluacion del modelo para el training set
            
            t=time.time()
            y_train_predict = lin_model.predict(X_train)
            elapsed=time.time()-t
            
            rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
            r2 = r2_score(Y_train, y_train_predict)
            
            print('The elapsed time was {}'.format(elapsed))
            print("The model performance for training set")
            print("--------------------------------------")
            print('RMSE is {}'.format(rmse))
            print('R2 score is {}'.format(r2))
            print("\n")
            
            # evaluacion del modelo para el test set
            
            y_test_predict = lin_model.predict(X_test)
            rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
            r2 = r2_score(Y_test, y_test_predict)
            
            print("The model performance for testing set")
            print("--------------------------------------")
            print('RMSE is {}'.format(rmse))
            print('R2 score is {}'.format(r2))
            return(r2)
        
        if tipo=='theta':
            
            Theta=[lin_model.intercept_,lin_model.coef_]
            return(Theta)
        
        if tipo=='pred':
            
            # make a prediction
            rta_new = lin_model.predict(X)
            rta_new=pd.DataFrame(rta_new, index=Caso_cero.index)
            comp_cero=pd.merge(rta_new, Y, on='Country Name', suffixes=('_left', '_right'))
            comp_cero.columns=['Predicción','GDP Real']
            return(comp_cero)
        
        if tipo=='modelo':
            
            return(lin_model)
        
caso_cero('stad')

The elapsed time was 0.0010330677032470703
The model performance for training set
--------------------------------------
RMSE is 0.07760211278207138
R2 score is 0.5826129369631897


The model performance for testing set
--------------------------------------
RMSE is 0.25210815265904324
R2 score is 0.21771747219555249


0.21771747219555249

In [34]:
caso_cero('theta') #Intercepto y coeficientes del modelo

[-0.07413780592915929,
 array([-0.07603141,  0.03729707,  0.17359076,  0.04252481,  0.30312677,
        -0.11219668,  0.0584731 ,  0.05096216,  0.15186152, -0.24404839,
         0.04214825])]

In [35]:
def caso_uno(tipo): 
    
        r_s=41
        #Caso uno
        Caso_uno=pd.merge(df_cluster_uno, Datos_pilar_infr, on='Country Name', suffixes=('_left', '_right'))
        Caso_uno=pd.merge(Caso_uno, df_GDP, on='Country Name', suffixes=('_left', '_right'))
        Caso_uno=Caso_uno.drop(['cluster'], axis=1)
        Caso_uno_array=Caso_uno.to_numpy()
        Caso_uno_array=(Caso_uno_array-np.min(Caso_uno_array))/(np.max(Caso_uno_array,axis=0)-np.min(Caso_uno_array))       #Normalizacion
        Caso_uno=pd.DataFrame(Caso_uno_array,index=Caso_uno.index)
        list_columns=lista_nombres_ind.copy()
        list_columns.append('GDP')
        Caso_uno.columns = list_columns
        X = Caso_uno.iloc[:,0:11]
        Y = Caso_uno['GDP']
        
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=r_s)
        lin_model1 = LinearRegression()
        lin_model1.fit(X_train, Y_train)
        
        if tipo=='graf':
            plt.scatter(X_train['Quality of road infrastructure'],Y_train)
            plt.xlabel("Quality of road infrastructure") # Establecer el título del eje x
            plt.ylabel("GDP") # Establecer el título del eje y
            
        if tipo=='stad':
            # Evaluacion del modelo para el training set
            t=time.time()
            y_train_predict = lin_model1.predict(X_train)
            elapsed=time.time()-t
            rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
            r2 = r2_score(Y_train, y_train_predict)
            print('The elapsed time was {}'.format(elapsed))
            print("The model performance for training set")
            print("--------------------------------------")
            print('RMSE is {}'.format(rmse))
            print('R2 score is {}'.format(r2))
            print("\n")
            # evaluacion del modelo para el test set
            y_test_predict = lin_model1.predict(X_test)
            rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
            r2 = r2_score(Y_test, y_test_predict)
            print("The model performance for testing set")
            print("--------------------------------------")
            print('RMSE is {}'.format(rmse))
            print('R2 score is {}'.format(r2))
            
        if tipo=='theta':
            Theta=[lin_model1.intercept_,lin_model1.coef_]
            return(Theta)
        
        if tipo=='pred':
            # make a prediction
            rta_new = lin_model1.predict(X)
            rta_new=pd.DataFrame(rta_new, index=Caso_uno.index)
            comp_uno=pd.merge(rta_new, Y, on='Country Name', suffixes=('_left', '_right'))
            comp_uno.columns=['Predicción','GDP Real']
            return(comp_uno)
        
        if tipo=='modelo':
            return(lin_model1)
        
caso_uno('stad')

The elapsed time was 0.0036270618438720703
The model performance for training set
--------------------------------------
RMSE is 0.02049250815658428
R2 score is 0.8012519374302236


The model performance for testing set
--------------------------------------
RMSE is 0.29398717913771466
R2 score is 0.00865374750549286


In [36]:
caso_uno('theta') #Intercepto y coeficientes del modelo

[0.058207586678601896,
 array([ 7.84402051e-02, -7.82421165e-02,  3.15717513e-02,  8.53178602e-03,
         1.99940656e-01,  7.92956866e-02, -3.75834542e-02, -9.98006870e-05,
        -2.25302240e-01, -5.94500016e-03, -9.36091329e-03])]

In [37]:
def caso_dos(tipo): 

        r_s=25
        #Caso dos
        Caso_dos=pd.merge(df_cluster_dos, Datos_pilar_infr, on='Country Name', suffixes=('_left', '_right'))
        Caso_dos=pd.merge(Caso_dos, df_GDP, on='Country Name', suffixes=('_left', '_right'))
        Caso_dos=Caso_dos.drop(['cluster'], axis=1)
        Caso_dos_array=Caso_dos.to_numpy()
        Caso_dos_array=(Caso_dos_array-np.min(Caso_dos_array))/(np.max(Caso_dos_array,axis=0)-np.min(Caso_dos_array))       #Normalizacion
        Caso_dos=pd.DataFrame(Caso_dos_array,index=Caso_dos.index)
        list_columns=lista_nombres_ind.copy()
        list_columns.append('GDP')
        Caso_dos.columns = list_columns
        X = Caso_dos.iloc[:,0:11]
        Y = Caso_dos['GDP']
        
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=r_s)
        lin_model2 = LinearRegression()
        lin_model2.fit(X_train, Y_train)
        
        if tipo=='graf':
            plt.scatter(X_train['Quality of road infrastructure'],Y_train)
            plt.xlabel("Quality of road infrastructure") # Establecer el título del eje x
            plt.ylabel("GDP") # Establecer el título del eje y
            
        if tipo=='stad':
            # Evaluacion del modelo para el training set
            t=time.time()
            y_train_predict = lin_model2.predict(X_train)
            elapsed=time.time()-t
            rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
            r2 = r2_score(Y_train, y_train_predict)
            print('The elapsed time was {}'.format(elapsed))
            print("The model performance for training set")
            print("--------------------------------------")
            print('RMSE is {}'.format(rmse))
            print('R2 score is {}'.format(r2))
            print("\n")
            # evaluacion del modelo para el test set
            y_test_predict = lin_model2.predict(X_test)
            rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
            r2 = r2_score(Y_test, y_test_predict)
            print("The model performance for testing set")
            print("--------------------------------------")
            print('RMSE is {}'.format(rmse))
            print('R2 score is {}'.format(r2))
            return(r2)
        if tipo=='theta':
            Theta=[lin_model2.intercept_,lin_model2.coef_]
            return(Theta)
        
        if tipo=='pred':
            # Realiza una predicion
            rta_new = lin_model2.predict(X)
            rta_new=pd.DataFrame(rta_new, index=Caso_dos.index)
            comp_dos=pd.merge(rta_new, Y, on='Country Name', suffixes=('_left', '_right'))
            comp_dos.columns=['Predicción','GDP Real']
            return(comp_dos)
        
        if tipo=='modelo':
            return(lin_model2)
caso_dos('stad')

The elapsed time was 0.001995563507080078
The model performance for training set
--------------------------------------
RMSE is 0.09238842453433099
R2 score is 0.78026418654369


The model performance for testing set
--------------------------------------
RMSE is 0.2114606557553321
R2 score is -24.308683405575973


-24.308683405575973

In [38]:
caso_dos('theta') #Intercepto y coeficientes del modelo

[-0.29777368083576294,
 array([-1.01329386,  0.09417108, -1.21808712, -0.13082012,  0.25157047,
         0.25841638,  0.47696053, -1.14915617,  1.35126889,  1.32189642,
        -0.02505249])]

<h3> Compare los parámetros de las tres hipótesis y concluya. Cómo afecta un cambio marginal en la
calidad de las vías (Quality of roads) el GDP de cada una de las categor´ıas de países? Nota: Este
análisis se debe hacer sobre la regresión obtenida para cada categoría. <h3>

In [39]:
#Aumento de 0.3
Datos_pilar_infr_n=np.zeros(len(Datos_pilar_infr))
for n in range (0,len(Datos_pilar_infr)):
    Datos_pilar_infr_n[n]=Datos_pilar_infr['Quality of road infrastructure'].iloc[n]*1.3
#Datos_pilar_infr_n

In [40]:
def aumento(caso):
    
    Datos_pilar_infr['Quality of road infrastructure']=Datos_pilar_infr_n
    
    if caso=='caso 0':
        Caso_cero=pd.merge(df_cluster_cero, Datos_pilar_infr, on='Country Name', suffixes=('_left', '_right'))
        Caso_cero=Caso_cero.drop(['cluster'], axis=1)
        Caso_cero_arrayn=Caso_cero.to_numpy()
        Caso_cero_arrayn=(Caso_cero_arrayn-np.min(Caso_cero_arrayn))/(np.max(Caso_cero_arrayn,axis=0)-np.min(Caso_cero_arrayn))       #Normalizacion
        Caso_ceron=pd.DataFrame(Caso_cero_arrayn,index=Caso_cero.index)
        rta_new=caso_cero('modelo').predict(Caso_cero)
        rta_new=pd.DataFrame(rta_new, index=Caso_cero.index)
        return(rta_new)
    
    if caso=='caso 1':
        Caso_uno=pd.merge(df_cluster_uno, Datos_pilar_infr, on='Country Name', suffixes=('_left', '_right'))
        Caso_uno=Caso_uno.drop(['cluster'], axis=1)
        Caso_uno_array=Caso_uno.to_numpy()
        Caso_uno_array=(Caso_uno_array-np.min(Caso_uno_array))/(np.max(Caso_uno_array,axis=0)-np.min(Caso_uno_array))       #Normalizacion
        Caso_uno=pd.DataFrame(Caso_uno_array,index=Caso_uno.index)
        X = Caso_uno
        rta_new=caso_uno('modelo').predict(X)
        rta_new=pd.DataFrame(rta_new, index=Caso_uno.index)
        return(rta_new)
    
    if caso=='caso 2':
        Caso_dos=pd.merge(df_cluster_dos, Datos_pilar_infr, on='Country Name', suffixes=('_left', '_right'))
        Caso_dos=Caso_dos.drop(['cluster'], axis=1)
        Caso_dos_array=Caso_dos.to_numpy()
        Caso_dos_array=(Caso_dos_array-np.min(Caso_dos_array))/(np.max(Caso_dos_array,axis=0)-np.min(Caso_dos_array))       #Normalizacion
        Caso_dos=pd.DataFrame(Caso_dos_array,index=Caso_dos.index)
        X = Caso_dos
        rta_new=caso_dos('modelo').predict(X)
        rta_new=pd.DataFrame(rta_new, index=Caso_dos.index)
        return(rta_new)

,0
Country Name,
Angola,0.035549
Albania,-0.012499
Burundi,-0.011303
Benin,-0.015671
Burkina Faso,0.003393
Bangladesh,0.247009
Bosnia and Herzegovina,-0.023787
Bolivia,-0.062069
Barbados,-0.131022


In [52]:
#Comparaciones

Comparacion_caso_cero=pd.merge(aumento('caso 0'), caso_cero('pred')['Predicción'], on='Country Name', suffixes=('_left', '_right'))
Comparacion_caso_cero.columns=['Aumentando la calidad de vías', 'Normal']

Comparacion_caso_uno=pd.merge(aumento('caso 1'), caso_uno('pred')['Predicción'], on='Country Name', suffixes=('_left', '_right'))
Comparacion_caso_uno.columns=['Aumentando la calidad de vías', 'Normal']

Comparacion_caso_dos=pd.merge(aumento('caso 2'), caso_dos('pred')['Predicción'], on='Country Name', suffixes=('_left', '_right'))
Comparacion_caso_dos.columns=['Aumentando la calidad de vías', 'Normal']

print('Caso cero: Aumentando la calidad de vías', Comparacion_caso_cero['Aumentando la calidad de vías'].mean(), 'Sin modificar', Comparacion_caso_cero['Normal'].mean(),'\n',
     'Caso uno: Aumentando la calidad de vías', Comparacion_caso_uno['Aumentando la calidad de vías'].mean(), 'Sin modificar', Comparacion_caso_uno['Normal'].mean(),'\n',
     'Caso dos: Aumentando la calidad de vías', Comparacion_caso_dos['Aumentando la calidad de vías'].mean(), 'Sin modificar', Comparacion_caso_dos['Normal'].mean())

Caso cero: Aumentando la calidad de vías -0.0008201995485222311 Sin modificar 0.07076931780023789 
 Caso uno: Aumentando la calidad de vías 0.027930138163521405 Sin modificar 0.027930138163521405 
 Caso dos: Aumentando la calidad de vías 0.08966663463064145 Sin modificar 0.0969304812796909


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


<h3>Construya una nueva regresión lineal que incluya todos los países. Compare el error de esta nueva
hipótesis con el de cada una de las tres categorías identificadas previamente.<\h3>

In [53]:
datos_completos=pd.merge(Datos_pilar_infr, df_GDP, on='Country Name', suffixes=('_left', '_right'))
datos_completos_Array=datos_completos.to_numpy()
datos_completos_Array=(datos_completos_Array-np.min(datos_completos_Array))/(np.max(datos_completos_Array,axis=0)-np.min(datos_completos_Array))       #Normalizacion
datos_completos=pd.DataFrame(datos_completos_Array,index=datos_completos.index)
list_columns=lista_nombres_ind.copy()
list_columns.append('GDP')
datos_completos.columns = list_columns
datos_completos

,Road connectivity,Quality of road infrastructure,Railroad density,Efficiency of train services,Airport connectivity,Efficiency of air transport services,Liner shipping connectivity,Efficiency of seaport services,Electricity supply quality,Exposure to unsafe drinking water,Reliability of water supply,GDP
Country Name,,,,,,,,,,,,
Angola,0.777352,0.211817,0.0,0.0,0.269248,0.424051,0.2513,0.334253,0.92434,0.233209,0.180303,0.003239
Albania,0.55524,0.526706,0.385949,0.03399,0.300837,0.565865,0.0304,0.602133,0.85056,0.833963,0.517318,0.000718
United Arab Emirates,0.900861,0.921892,0.0,0.0,0.891738,0.929264,0.8391,0.817733,0.967272,0.901233,0.855875,0.019547
Argentina,0.945312,0.479986,0.16086,0.293116,0.590523,0.635545,0.3519,0.530717,0.905881,0.93641,0.731455,0.020939
Armenia,0.58624,0.487037,0.60209,0.365419,0.332327,0.669917,0.0,0.260566,0.926053,0.988577,0.701977,0.000634
...,...,...,...,...,...,...,...,...,...,...,...,...
Uruguay,0.898031,0.498479,0.213976,0.037268,0.28672,0.767989,0.3319,0.694503,0.907484,0.962521,0.866102,0.0029
United States,1.0,0.822992,0.412592,0.721276,1.0,0.884917,0.9666,0.835574,0.985853,1.0,0.869268,1.0
South Africa,0.962064,0.65278,0.431811,0.355019,0.634658,0.828354,0.4011,0.650509,0.951331,0.736404,0.578307,0.018169


In [56]:
X = datos_completos.iloc[:,0:11]
Y = datos_completos['GDP']
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

# Evaluacion del modelo para el training set
t=time.time()
y_train_predict = lin_model.predict(X_train)
elapsed=time.time()-t
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)
print('The elapsed time was {}'.format(elapsed))
print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")
# evaluacion del modelo para el test set
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)
print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print([lin_model.intercept_,lin_model.coef_])

The elapsed time was 0.001996755599975586
The model performance for training set
--------------------------------------
RMSE is 0.023024328806914383
R2 score is 0.5388682944240104


The model performance for testing set
--------------------------------------
RMSE is 0.22017894161036186
R2 score is 0.03906096948820725
[0.07627755574373342, array([-0.00584635,  0.00082343,  0.01630934,  0.02143144,  0.0934974 ,
        0.00033216,  0.00951854, -0.0144179 , -0.11155116, -0.01106232,
        0.00206471])]
